# Sync Bullet Overflow

Sync vault note tracking overflow tasks in bullet list.

- https://markdown-it-py.readthedocs.io/en/latest/using.html#markdown-renderer
- https://mdformat.readthedocs.io/en/stable/users/installation_and_usage.html

In [ ]:
from pathlib import Path
from subprocess import run

from mdformat.renderer import MDRenderer
from mdformat_frontmatter import plugin
from more_itertools import one
from pandas import DataFrame, read_csv

from notes import bullet
from notes.datetime import current_tz
from notes.markdown import MD

DEBUG = False
DAY = "2025-05-05"
BULLET = (
    bullet.PATH
    if DEBUG
    else Path(f"data/local/vaults/personal/_data/bullet_export_{DAY}.csv")
)
NOTE = Path(
    "D:/code/mine/notes/data/local/vaults/personal/_logs/bullet-list/current.md"
)
if DEBUG:
    run(args=["uv", "run", "-m", "notes.bullet"], check=True)

bullets = read_csv(BULLET, engine="pyarrow").query("type == 'task'")
tokens = MD.parse(NOTE.read_text(encoding="utf-8"))

bullets_preview_ = bullets.dropna(axis="columns", how="all")
tokens_preview_ = DataFrame(tokens)

text = "\n\n".join([
    MDRenderer().render(
        tokens=tokens[
            : tokens.index(
                one(t for t in tokens if t.type == "heading_open" and t.tag == "h1")
            )
        ],
        options={"parser_extension": [plugin]},
        env={},
        finalize=False,
    ),
    f"# Current ({DAY})",
    "## To-do",
    bullets.query("list_title == 'To-do' & date_complete.isna()").pipe(
        lambda df: "\n".join(
            f"- [ ] {title}{'' if '🆔' in title else ' 🆔 ' + date_created.astimezone(current_tz).isoformat(timespec='seconds').replace(':', '')}"
            for title, date_created in zip(
                df["title"], df["date_created"], strict=False
            )
            if date_created
        )
    ),
    "## Overflow",
    bullets.query("list_title == 'Overflow'").pipe(
        lambda df: "\n".join(
            f"- [ ] {title} 🆔 {id}"
            for title, id in zip(df["title"], df["id"], strict=False)  # noqa: A001
        )
    ),
    "## Complete",
    bullets.query("list_title == 'To-do' & date_complete.notna()").pipe(
        lambda df: "\n".join(
            f"- [x] {title}{'' if '🆔' in title else ' 🆔 ' + date_created.astimezone(current_tz).isoformat(timespec='seconds').replace(':', '')} ✅ {date_complete.astimezone(current_tz).date().isoformat()}"
            for title, date_created, date_complete in zip(
                df["title"], df["date_created"], df["date_complete"], strict=False
            )
        )
    ),
])

NOTE.write_text(encoding="utf-8", data=text)
print(text)